# Calculate Simplified Ikeda simulation

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 7

from rolldecayestimators.simplified_ikeda import calculate_roll_damping
from rolldecayestimators import equations
import sympy as sp
from rolldecayestimators import symbols
from rolldecayestimators.substitute_dynamic_symbols import lambdify
from mdldb.tables import Run
from rolldecayestimators.direct_linear_estimator import DirectLinearEstimator
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from sklearn.pipeline import Pipeline

import data
import copy
from rolldecay import database
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

In [ ]:
db = database.get_db()

In [ ]:
required = ['lpp','beam','TA','TF','BKL','BKB','omega0','kg','Volume','A0']

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_linear')

df_rolldecay.fillna(0, inplace=True)
mask = ((df_rolldecay['ship_speed']==0) &  ## Zero speed!
        (df_rolldecay[required] > 0).all(axis=1)
       )

df = df_rolldecay.loc[mask].copy()
df['phi_max'] = np.rad2deg(df['phi_start'].abs())

In [ ]:
row = df.iloc[0]

In [ ]:
db_run = db.session.query(Run).get(int(row.id))

In [ ]:
df = database.load_run(db_run=db_run)

In [ ]:
df.plot(y='phi')

In [ ]:
def calculate(row):
    
    LPP = row.lpp
    Beam = row.beam
    DRAFT = (row.TA + row.TF)/2
    
    PHI = row.phi_max
    lBK = row.BKL
    bBK = row.BKB
    OMEGA = row.omega0
    OG = (-row.kg+DRAFT)
    CB = row.Volume/(row.lpp*row.beam*DRAFT)
    CMID = row.A0
    
    B44HAT, BFHAT, BWHAT, BEHAT, BBKHAT = calculate_roll_damping(LPP,Beam,CB,CMID,OG,PHI,lBK,bBK,OMEGA,DRAFT)
    s = pd.Series()
    s['B44HAT'] = B44HAT
    s['BFHAT'] = BFHAT
    s['BWHAT'] = BWHAT
    s['BEHAT'] = BEHAT
    s['BBKHAT'] = BBKHAT
    return s
    

In [ ]:
result = calculate(row)
result

In [ ]:
Math(vlatex(equations.B44_equation))

In [ ]:
sp.Eq(symbols.B_44, sp.solve(equations.B44_equation, symbols.B_44)[0])

In [ ]:
B44_lambda = lambdify(sp.solve(equations.B44_equation, symbols.B_44)[0])

In [ ]:
g=9.81
rho=1000
B_44 = B44_lambda(B_44_hat=result.B44HAT, Disp=row.Volume, beam=row.beam, g=g, rho=rho)

Equivalent linearized damping according to [Himeno]:

In [ ]:
equations.B_e_equation

In [ ]:
equations.zeta_equation

In [ ]:
equations.omega0_equation_linear

In [ ]:
eqs = [equations.zeta_equation,
      equations.omega0_equation_linear]

In [ ]:
zeta_lambda = lambdify(sp.solve(eqs,symbols.A_44,symbols.zeta)[0][1])

In [ ]:
m = row.Volume*rho
zeta=zeta_lambda(B_1=B_44, GM=row.gm, g=g, m=m,  omega0=row.omega0)
zeta

In [ ]:
row.zeta

In [ ]:
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=2, minimum_score=0.99)
scaler = ScaleFactorTransformer(scale_factor=None)  # dummy value None for now
cutter = CutTransformer(phi_max=np.deg2rad(9), phi_min=np.deg2rad(0.25))
offset_transformer = OffsetTransformer()

In [ ]:
scaler.scale_factor=row.scale_factor

In [ ]:
direct_linear_estimator = DirectLinearEstimator()

steps = [
    ('filter',lowpass_filter),
    ('scaler',scaler),
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer),]
    #('linear_estimator', direct_linear_estimator)]
       
preprocessor = Pipeline(steps) # define the pipeline object.

In [ ]:
preprocessor.fit(df)
X=preprocessor.transform(df)

In [ ]:
X.plot(y='phi')

In [ ]:
estimator = DirectLinearEstimator()
t=X.index
phi0=X.iloc[0]['phi']
phi1d0=X.iloc[0]['phi1d']

df_sim_ikeda = estimator.simulate(t=t, phi0=phi0, phi1d0=phi1d0, omega0=row.omega0, zeta=zeta)
df_sim = estimator.simulate(t=t, phi0=phi0, phi1d0=phi1d0, omega0=row.omega0, zeta=row.zeta)


In [ ]:
df_result = X.copy()
df_result = pd.merge(left=df_result, right=df_sim_ikeda, how='left', left_index=True, right_index=True, 
                     suffixes =('','_ikeda'))

df_result = pd.merge(left=df_result, right=df_sim, how='left', left_index=True, right_index=True, 
                     suffixes =('','_model'))

fig,ax=plt.subplots()
df_result.plot(y='phi',label='model test', style=':', ax=ax)
df_result.plot(y='phi_model',label='model', style='-',ax=ax)
df_result.plot(y='phi_ikeda',label='ikeda', style='--',ax=ax)